In [1]:
# install necessary packages to access google drive.
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

!mkdir -p drive
!google-drive-ocamlfuse drive

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package libfuse2:amd64.
(Reading database ... 22280 files and directories currently installed.)
Preparing to unpack .../libfuse2_2.9.7-1ubuntu1_amd64.deb ...
Unpacking libfuse2:amd64 (2.9.7-1ubuntu1) ...
Selecting previously unselected package fuse.
Preparing to unpack .../fuse_2.9.7-1ubuntu1_amd64.deb ...
Unpacking fuse (2.9.7-1ubuntu1) ...
Selecting previously unselected package google-drive-ocamlfuse.
Preparing to unpack .../google-drive-ocamlfuse_0.7.0-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.0-0ubuntu1~ubuntu18.04.1) ...
Setting up libfuse2:amd64 (2.9.7-1ubuntu1) ...
Processing triggers for libc-bin (2.27-3ubuntu1) ...
Setting up fuse (2.9.7-1ubuntu1) ...
Setting up google-drive-ocamlfuse (0.7.0-0ubuntu1~ubuntu18.04.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleus

In [2]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import seaborn as sns 
import matplotlib.pyplot as plt
import math
from itertools import chain

data_dir = "drive/5001/project/"
print(os.listdir(data_dir))

['train_V2.csv.zip']


## 0. Initial

In [0]:
data = pd.read_csv(data_dir + 'train_V2.csv.zip')
# print(len(data))

In [4]:
print(len(data))

4446966


In [0]:
data.columns.values

array(['Id', 'groupId', 'matchId', 'assists', 'boosts', 'damageDealt',
       'DBNOs', 'headshotKills', 'heals', 'killPlace', 'killPoints',
       'kills', 'killStreaks', 'longestKill', 'matchDuration',
       'matchType', 'maxPlace', 'numGroups', 'rankPoints', 'revives',
       'rideDistance', 'roadKills', 'swimDistance', 'teamKills',
       'vehicleDestroys', 'walkDistance', 'weaponsAcquired', 'winPoints',
       'winPlacePerc'], dtype=object)

In [0]:
# df_train.hist(bins=10, figsize=(30, 30))

## 1.1. First step of Filtering 

In [3]:
relevant = data[data.isnull().values==True]['groupId'].values
print(len(relevant),'groups')

1 groups


In [4]:
for group in relevant:
#     print(len(data[data['groupId'] == group]))
    data.drop(data[data['groupId'] == group].index, inplace=True)
print(len(data))

4446965


In [0]:
data['matchType'].value_counts()

squad-fpp           1756186
duo-fpp              996691
squad                626526
solo-fpp             536762
duo                  313591
solo                 181943
normal-squad-fpp      17174
crashfpp               6287
normal-duo-fpp         5489
flaretpp               2505
normal-solo-fpp        1682
flarefpp                718
normal-squad            516
crashtpp                371
normal-solo             326
normal-duo              199
Name: matchType, dtype: int64

In [5]:
data.drop(data[data['matchType'] == 'normal-squad-fpp'].index, inplace=True)
data.drop(data[data['matchType'] == 'crashfpp'].index, inplace=True)
data.drop(data[data['matchType'] == 'normal-duo-fpp '].index, inplace=True)
data.drop(data[data['matchType'] == 'flaretpp'].index, inplace=True)
data.drop(data[data['matchType'] == 'normal-solo-fpp'].index, inplace=True)
data.drop(data[data['matchType'] == 'flarefpp'].index, inplace=True)
data.drop(data[data['matchType'] == 'normal-squad'].index, inplace=True)
data.drop(data[data['matchType'] == 'crashtpp'].index, inplace=True)
data.drop(data[data['matchType'] == 'normal-solo'].index, inplace=True)
data.drop(data[data['matchType'] == 'normal-duo '].index, inplace=True)
print(len(data))

4417386


In [0]:
# from tqdm._tqdm_notebook import tnrange

In [6]:
temp = data[data['matchType'] == 'solo']
group_data = temp.groupby(['groupId'])
stat = group_data['Id'].count()
stat = pd.DataFrame(stat)
relevant = stat[stat['Id'] > 1].index.values
print(len(relevant),'groups')

5598 groups


In [0]:
# data.reset_index()

In [0]:
indexes_to_keep = set(range(data.shape[0])) #- set(indexes_to_drop)

In [0]:

for group in relevant:
#     print(len(data[data['groupId'] == group]))
#     data.drop(data[data['groupId'] == group].index, inplace=True)
  indexes_to_keep -= set(data[data['groupId'] == group].index)
# print(len(data)

In [14]:
data = data.take(list(indexes_to_keep))
print(len(data))


4398541


In [0]:
data.to_csv('drive/temp1.csv')

In [0]:
# data = pd.read_csv('drive/temp1.csv')

In [16]:
temp = data[data['matchType'] == 'solo-fpp']
group_data = temp.groupby(['groupId'])
stat = group_data['Id'].count()
stat = pd.DataFrame(stat)

relevant = stat[stat['Id'] > 1].index.values
print(len(relevant),'groups')

11376 groups


In [0]:
data.reset_index()
indexes_to_keep = set(range(data.shape[0]))

In [0]:
# max = 0
for group in relevant[1:]:
#     print(len(data[data['groupId'] == group]))
# #     if len(data[data['groupId'] == group]) > max:
#         max = len(data[data['groupId'] == group])
#     data.drop(data[data['groupId'] == group].index, inplace=True)
  indexes_to_keep -= set(data[data['groupId'] == group].index)
# print(len(data))

In [22]:
data = data.take(list(indexes_to_keep))
print(len(data))

4374267


In [0]:
data.to_csv('drive/temp2.csv')

In [0]:
data = pd.read_csv('drive/temp2.csv')

In [0]:
print(len(data))

4374267


## 1.2. Second step of Filtering 

In [4]:
relevant = data[data['longestKill'] >= 1000]['groupId'].values
print(len(relevant),'groups')
for group in relevant:
#     print(len(data[data['groupId'] == group]))
    data.drop(data[data['groupId'] == group].index, inplace=True)
print(len(data))

6 groups
4374251


In [5]:
relevant = data[data['roadKills'] > 10]['groupId'].values
print(len(relevant),'groups')
for group in relevant:
#     print(len(data[data['groupId'] == group]))
    data.drop(data[data['groupId'] == group].index, inplace=True)
print(len(data))

4 groups
4374247


In [0]:
#We try to identify cheaters by checking if people are getting kills without moving. 
data['totalDistance'] = data['rideDistance'] + data['walkDistance'] + data['swimDistance']
data['killsWithoutMoving'] = ((data['kills'] > 0) & (data['totalDistance'] == 0))

In [7]:
relevants = []
relevants.append(list(data[data['walkDistance'] >= 10000]['groupId'].values))
relevants.append(list(data[data['rideDistance'] >= 20000]['groupId'].values))
relevants.append(list(data[data['swimDistance'] >= 2000]['groupId'].values))
relevants.append(list(data[data['killsWithoutMoving'] == True]['groupId'].values))
relevants = list(chain.from_iterable(relevants))
print(len(relevants),'groups')

675 groups


In [0]:
data.reset_index()
indexes_to_keep = set(range(data.shape[0]))

In [9]:
for group in relevants:
#     print(len(data[data['groupId'] == group]))
#     data.drop(data[data['groupId'] == group].index, inplace=True)
   indexes_to_keep -= set(data[data['groupId'] == group].index)
print(len(data))

4374247


In [10]:
data = data.take(list(indexes_to_keep))
print(len(data))

4373061


In [0]:
#The feature headshot_rate will also help us to catch cheaters.
# Create headshot_rate feature
data['headshot_rate'] = data['headshotKills'] / data['kills']
data['headshot_rate'] = data['headshot_rate'].fillna(0)

In [12]:
relevant = data[(data['headshot_rate']==1) & (data['kills']>9)]['groupId'].values
print(len(relevant),'groups')

21 groups


In [13]:
for group in relevant:
#     print(len(data[data['groupId'] == group]))
    data.drop(data[data['groupId'] == group].index, inplace=True)
print(len(data))

4373006


In [14]:
relevant = data[data['weaponsAcquired'] >= 80]['groupId'].values
print(len(relevant),'groups')
for group in relevant:
#     print(len(data[data['groupId'] == group]))
    data.drop(data[data['groupId'] == group].index, inplace=True)
print(len(data))

14 groups
4372962


In [15]:
data.reset_index()
indexes_to_keep = set(range(data.shape[0]))

relevant = data[data['heals'] >= 40]['groupId'].values
print(len(relevant),'groups')
for group in relevant:
#     print(len(data[data['groupId'] == group]))
#     data.drop(data[data['groupId'] == group].index, inplace=True)
  indexes_to_keep -= set(data[data['groupId'] == group].index)
# print(len(data))

133 groups


In [16]:
data = data.take(list(indexes_to_keep))
print(len(data))

4372756


In [0]:
data.to_csv('drive/temp3.csv')

## 2. Group by data by group & match 

In [0]:
agg_funcs = {'assists':'sum',
             'boosts':'sum',
             'damageDealt':'sum',
             'DBNOs':'sum',
             'headshotKills':'sum',
             'heals':'sum',
             'killPlace':'min',
             'killPoints':'sum',
             'kills':'sum',
             'killStreaks':'sum',
             'longestKill':'max',
             'matchDuration':'mean',
             'matchType':'max', # new value
             'maxPlace':'max', # may not equal to group num
             'numGroups':'max',
             'rankPoints':'sum', # depatured in next version
             'revives':'sum',
             'rideDistance':'sum',
             'roadKills':'sum',
             'swimDistance':'sum',
             'teamKills':'sum',
             'vehicleDestroys':'sum',
             'walkDistance':'sum',
             'weaponsAcquired':'sum',
             'winPoints':'sum', # rankPoints where only win counts
             'winPlacePerc':'max'} # 1 is best, 0 is last

In [0]:
# group_match_data = data.groupby(['groupId', 'matchId'])

In [0]:
group_data = data.groupby(['groupId'])

In [0]:
group_data = group_data.agg(agg_funcs)

In [0]:
# group_match_data = group_match_data.agg(agg_funcs)
# group_match_data = group_match_data.sort_values(by=['matchId','winPlacePerc'],
#                                                 ascending=False)
# group_match_data = group_match_data.reset_index()
# group_match_data = group_match_data.sort_values(by=['matchId', 'winPlacePerc'],
#                                                 ascending=False)

In [27]:
len(group_data)

2008933

In [26]:
group_data.head()

,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,killPoints,kills,killStreaks,...,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,winPlacePerc
groupId,,,,,,,,,,,,,,,,,,,,,
00000c08b5be36,0,1,741.50,5,1,1,35,0,2,2,...,0,0.0,0,0.00,0,0,814.60,7,0,0.2000
00000d1cbbc340,0,0,173.70,0,0,0,40,0,1,1,...,0,0.0,0,0.00,0,0,135.80,3,0,0.1154
000025a09dd1d7,0,0,0.00,0,0,0,72,0,0,0,...,0,0.0,0,0.00,0,0,110.80,2,0,0.2000
000038ec4dff53,1,5,211.57,1,1,7,9,0,3,2,...,2,0.0,0,152.09,0,0,4185.00,10,0,0.9615
00003a54230763,1,0,100.00,1,1,0,34,2026,1,1,...,0,0.0,0,0.00,0,0,185.35,5,3002,0.1395


In [0]:
group_data.to_csv('drive/group_data.csv')

**save group match data. because groupby is very slow.**

In [29]:
group_data.columns.values

array(['assists', 'boosts', 'damageDealt', 'DBNOs', 'headshotKills',
       'heals', 'killPlace', 'killPoints', 'kills', 'killStreaks',
       'longestKill', 'matchDuration', 'matchType', 'maxPlace',
       'numGroups', 'rankPoints', 'revives', 'rideDistance', 'roadKills',
       'swimDistance', 'teamKills', 'vehicleDestroys', 'walkDistance',
       'weaponsAcquired', 'winPoints', 'winPlacePerc'], dtype=object)

## Preparation for Machine Learning 

In [0]:
train = group_data[group_data['matchType'] == 'solo']

In [0]:
from sklearn.ensemble import RandomForestRegressor

regr = RandomForestRegressor(max_depth=8, random_state=0, n_estimators=100)


In [0]:
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_predict

In [0]:
train.drop(['matchType'],1,inplace=True)

In [41]:
prediction = cross_val_predict(regr, train.values[:, :-1], train.values[:, -1], cv=10)

print("使用交叉验证的误差为:",mean_absolute_error(train.values[:, -1], prediction))

使用交叉验证的误差为: 0.058255162646404644
